In [1]:
from sqlalchemy import *
from sqlalchemy.orm import *
import pandas as pd
import random
import numpy as np
import os
from sqlalchemy.event import *
from gradedb import GradeDB

In [2]:
%run schema.py
db=GradeDB("create.db")
db.Create()

In [3]:
first_names=('John','Andy','Joe','Felix','Alice','April','Reiter',
             'Emory','Miller','David','Ballin','Trotter', 'Virginia',
             'Rios','Thomas','Wheeler','James','Harrell', 'Nicolas',
             'White','Mary','Flanagan','Velda','Kitty','Iris','Violet',
            'Kara','Betty','Lilly','Troy','Crystal','Cory','Leila',
            'Kristina','Lena','Hanna','Josie','Jean','Ellie','Aliya',
            'Max','Cora','Meghan','Lucie','Ann','Orla','Lana','Tilly')
last_names=('Johnson','Smith','Williams','Hill','Morton','Hutchinson',
           'Brooks','Jensen','Rogers','Montgomery','Booth','Pham','Austin',
           'Day','Ayala','Page','Wang','Rice','Vazquez','Wolf','Abbott',
           'Mcguire','Aguilar','Henry','Ross','Cole','Pratt','Frank','Cummings',
           'Clark','Wilson','Hammond','Porter','Robles','Beck','Harvey','Benson',
           'Franklin','Bass','Lloyd','Allen','Flores','Harris','Young',
           'Bell','Miller','Stevens','Williamson','Lynch','Gough','Waters')

student_names = []
np.random.seed(123)
for i in range(100):
    full_name=np.random.choice(first_names)+' '+random.choice(last_names)
    student_names.append(full_name)
student_names=tuple(student_names)

np.random.seed(123)
s_num=np.random.randint(3000000,3099999,size=100)
s_num=s_num.astype(str)
uid=[]
email=[]
for item in s_num:
    uid.append('s'+item)
    email.append('s'+item+'@leidenuniv.nl')
uid=tuple(uid)
email=tuple(email)

for i in range(100):
    db.addStudent(uid[i],student_names[i],email[i])

In [4]:
#view student
engine0 = create_engine("sqlite:///create.db",echo=False)
session = Session(engine0)
stmt = select(Students)

for user in session.scalars(stmt):
    print(user)

Students(Universityid='s3015725', Name='Orla Miller', Email='s3015725@leidenuniv.nl')
Students(Universityid='s3028030', Name='Joe Williamson', Email='s3028030@leidenuniv.nl')
Students(Universityid='s3017730', Name='Lilly Miller', Email='s3017730@leidenuniv.nl')
Students(Universityid='s3061404', Name='Lena Austin', Email='s3061404@leidenuniv.nl')
Students(Universityid='s3063206', Name='Ellie Miller', Email='s3063206@leidenuniv.nl')
Students(Universityid='s3015377', Name='Harrell Allen', Email='s3015377@leidenuniv.nl')
Students(Universityid='s3073299', Name='White Gough', Email='s3073299@leidenuniv.nl')
Students(Universityid='s3046203', Name='Meghan Rice', Email='s3046203@leidenuniv.nl')
Students(Universityid='s3089302', Name='Velda Ross', Email='s3089302@leidenuniv.nl')
Students(Universityid='s3022241', Name='Kristina Cole', Email='s3022241@leidenuniv.nl')
Students(Universityid='s3065632', Name='Leila Robles', Email='s3065632@leidenuniv.nl')
Students(Universityid='s3055409', Name='Tilly

In [5]:
def Q_generator(size=100,s=123):
    '''default: call this function return a question df with column called Title, Text'''
    '''default: 100 rows, seed 123'''
    np.random.seed(s)
    r=np.arange(1,size+1,1).astype(str)
    r=np.random.choice(r,size,replace=False)
    Question=pd.DataFrame(columns=['Index','Title','Text'])
    title=[]
    text=[]
    index=[]
    for i in range(len(r)):
        index.append(r[i])
        title.append('Question Title '+r[i])
        text.append('Example question text '+r[i]+': A teacher adds ......')
    Question['Index']=index
    Question['Title']=title
    Question['Text']=text
    return(Question)

In [6]:
Q=Q_generator()
for i in range(len(Q)):
    db.addQuestion(Q.loc[i,'Index'],Q.loc[i,'Title'],Q.loc[i,'Text'])

In [7]:
#view question
#engine0 = create_engine("sqlite:///create.db",echo=False)
#session = Session(engine0)
stmt = select(Question)

for user in session.scalars(stmt):
    print(user)

Question(Questionid='1', Title='Question Title 1', Text='Example question text 1: A teacher adds ......')
Question(Questionid='2', Title='Question Title 2', Text='Example question text 2: A teacher adds ......')
Question(Questionid='3', Title='Question Title 3', Text='Example question text 3: A teacher adds ......')
Question(Questionid='4', Title='Question Title 4', Text='Example question text 4: A teacher adds ......')
Question(Questionid='5', Title='Question Title 5', Text='Example question text 5: A teacher adds ......')
Question(Questionid='6', Title='Question Title 6', Text='Example question text 6: A teacher adds ......')
Question(Questionid='7', Title='Question Title 7', Text='Example question text 7: A teacher adds ......')
Question(Questionid='8', Title='Question Title 8', Text='Example question text 8: A teacher adds ......')
Question(Questionid='9', Title='Question Title 9', Text='Example question text 9: A teacher adds ......')
Question(Questionid='10', Title='Question Titl

In [8]:
def T_generator(qid,size=100,question_num=7,s=123):
    '''qid represents for all the question id in the database'''
    '''default: call this function return a task df with column called Title, Text'''
    '''default: 100 rows, seed 123'''
    '''default: every task has 7 questions'''
    np.random.seed(s)
    r=np.random.randint(1,100,size).astype(str)
    Task=pd.DataFrame(columns=['Title','Text','qid_list'])
    title=[]
    text=[]
    qid_list=[]
    for i in range(len(r)):
        title.append('Task Title '+r[i])
        text.append('Example Task text '+r[i]+': This task should ......')
        qid_list.append(np.random.choice(qid,size=question_num,replace=False))
    Task['Title']=title
    Task['Text']=text
    Task['qid_list']=qid_list
    return(Task)

In [9]:
Task_Gen=T_generator(qid=Q["Index"])

for i in range(len(Task_Gen)):
    db.addTask(Task_Gen.loc[i,"Title"],Task_Gen.loc[i,"Text"],i,Task_Gen.loc[i,'qid_list'])

In [10]:
#view task
#engine0 = create_engine("sqlite:///create.db",echo=False)
#session = Session(engine0)
stmt = select(Task)

for user in session.scalars(stmt):
    print(user)

Students(Taskid='0', Title='Task Title 67', Text='Example Task text 67: This task should ......')
Students(Taskid='1', Title='Task Title 93', Text='Example Task text 93: This task should ......')
Students(Taskid='2', Title='Task Title 99', Text='Example Task text 99: This task should ......')
Students(Taskid='3', Title='Task Title 18', Text='Example Task text 18: This task should ......')
Students(Taskid='4', Title='Task Title 84', Text='Example Task text 84: This task should ......')
Students(Taskid='5', Title='Task Title 58', Text='Example Task text 58: This task should ......')
Students(Taskid='6', Title='Task Title 87', Text='Example Task text 87: This task should ......')
Students(Taskid='7', Title='Task Title 98', Text='Example Task text 98: This task should ......')
Students(Taskid='8', Title='Task Title 97', Text='Example Task text 97: This task should ......')
Students(Taskid='9', Title='Task Title 48', Text='Example Task text 48: This task should ......')
Students(Taskid='10'

In [11]:
#view relation Question_Task
#engine0 = create_engine("sqlite:///create.db",echo=False)
#session = Session(engine0)
stmt = select(Question_Task)

for user in session.scalars(stmt):
    print(user)

28
64
40
84
26
35
47
52
28
33
97
19
94
20
24
17
66
20
90
70
60
73
81
26
60
18
10
1
28
66
84
93
32
23
71
80
87
100
69
92
33
40
51
65
77
52
19
33
42
2
43
58
51
37
76
27
25
50
72
49
100
93
90
12
52
92
45
67
24
55
48
21
86
80
91
38
69
9
68
30
91
44
72
10
93
90
39
63
91
60
43
84
89
32
57
24
91
42
66
76
85
47
31
11
25
85
50
23
33
61
60
15
50
91
68
45
9
96
37
97
23
16
98
31
45
50
64
71
57
85
98
20
15
79
38
26
5
73
12
3
51
27
67
25
6
47
76
6
43
3
88
70
99
93
37
32
21
29
45
7
2
47
27
2
61
70
95
26
13
4
80
59
19
54
14
86
55
57
100
48
78
6
22
48
18
41
40
61
13
79
56
73
100
17
16
53
93
15
37
99
43
35
71
92
17
45
5
23
11
61
30
9
34
17
12
15
41
71
25
84
34
22
60
48
4
32
79
65
31
96
43
46
71
10
83
100
81
89
24
91
37
31
55
6
2
13
40
78
23
94
24
60
25
97
73
1
70
94
29
61
70
54
68
65
75
44
89
5
2
46
76
68
31
88
34
71
86
59
2
43
31
50
37
33
35
74
10
24
39
53
25
30
14
78
48
40
46
63
87
91
20
18
20
70
92
55
4
8
67
63
56
21
23
61
6
83
85
47
8
12
59
36
61
84
62
88
99
17
35
9
15
62
64
42
77
33
83
77
58
42
49


In [12]:
for i in range(100):
    db.addAssignment(taskid=i,universityid=uid[i])

In [13]:
#view assignment
#engine0 = create_engine("sqlite:///create.db",echo=False)
#session = Session(engine0)
stmt = select(Assignment)

for user in session.scalars(stmt):
    print(user)
    #db.New_Ass_Remind()

Assignment(Universityid='s3015725', Taskid='0',Assignmentid='1')
Assignment(Universityid='s3028030', Taskid='1',Assignmentid='2')
Assignment(Universityid='s3017730', Taskid='2',Assignmentid='3')
Assignment(Universityid='s3061404', Taskid='3',Assignmentid='4')
Assignment(Universityid='s3063206', Taskid='4',Assignmentid='5')
Assignment(Universityid='s3015377', Taskid='5',Assignmentid='6')
Assignment(Universityid='s3073299', Taskid='6',Assignmentid='7')
Assignment(Universityid='s3046203', Taskid='7',Assignmentid='8')
Assignment(Universityid='s3089302', Taskid='8',Assignmentid='9')
Assignment(Universityid='s3022241', Taskid='9',Assignmentid='10')
Assignment(Universityid='s3065632', Taskid='10',Assignmentid='11')
Assignment(Universityid='s3055409', Taskid='11',Assignmentid='12')
Assignment(Universityid='s3023166', Taskid='12',Assignmentid='13')
Assignment(Universityid='s3071200', Taskid='13',Assignmentid='14')
Assignment(Universityid='s3033710', Taskid='14',Assignmentid='15')
Assignment(Uni

In [14]:
for i in range(100):
    db.newSubmission(universityid = uid[i])


C:\Users\Administrator\Desktop\pywork\DB\gradedb.py:73: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  a = Submission(Evaluation_request=False,Time=pd.datetime.now())


In [15]:
#view newsub
engine0 = create_engine("sqlite:///create.db",echo=False)
session = Session(engine0)
stmt = select(Submission)

for user in session.scalars(stmt):
    print(user)

Submission(Submissionid='1', Assignmentid='1',Time='2022-04-11 14:23:02.930362', Evaluation='None',  Evaluation_request='False',Evaluation_finished='False')
Submission(Submissionid='2', Assignmentid='2',Time='2022-04-11 14:23:02.953510', Evaluation='None',  Evaluation_request='False',Evaluation_finished='False')
Submission(Submissionid='3', Assignmentid='3',Time='2022-04-11 14:23:02.966363', Evaluation='None',  Evaluation_request='False',Evaluation_finished='False')
Submission(Submissionid='4', Assignmentid='4',Time='2022-04-11 14:23:02.984207', Evaluation='None',  Evaluation_request='False',Evaluation_finished='False')
Submission(Submissionid='5', Assignmentid='5',Time='2022-04-11 14:23:03.003481', Evaluation='None',  Evaluation_request='False',Evaluation_finished='False')
Submission(Submissionid='6', Assignmentid='6',Time='2022-04-11 14:23:03.014731', Evaluation='None',  Evaluation_request='False',Evaluation_finished='False')
Submission(Submissionid='7', Assignmentid='7',Time='2022-0

In [16]:
def Ans_generator(size=100,s=123):
    np.random.seed(s)
    r=np.random.randint(1,100,size).astype(str)
    Answer=pd.DataFrame(columns=['id','Text'])
    id=[]
    text=[]
    for i in range(len(r)):
        id.append(r[i])
        text.append('Example Answer text '+r[i]+': My solution is ......')
    Answer['id']=id
    Answer['Text']=text
    return(Answer)

Ans=Ans_generator()

In [17]:
for i in range(90):
    db.addAnswer(Q_id=[1,2,3,4,5,6,7],universityid=uid[i],text=Ans.loc[i][1])


In [18]:
def Score_generator(mean=6,sd=3,size=100,n=7,s=123):
    '''default: call this function return 1 random num from Normal(6,9)'''
    '''defult random seed = 123'''
    '''every student has 7 question answers'''
    np.random.seed(s)
    r=np.random.normal(mean,sd,(100,7))
    for i in range(len(r)):
        for j in range(r.shape[1]):
            if r[i,j]>10:
                r[i,j]=round(10,1)
            elif r[i,j]<0:
                r[i,j]=round(0,1)
            else:
                r[i,j]=round(r[i,j],1)
    return r

ran_score=Score_generator()

In [19]:
for i in range(len(uid)):
    db.addScore(uid[i],ran_score[i])

C:\Users\Administrator\Desktop\pywork\DB\gradedb.py:115: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  a = Score(Submissionid=submissionid,Answerid=answeridlist[i],Value=value[i],Time=pd.datetime.now())


In [20]:
#view newsub
engine0 = create_engine("sqlite:///create.db",echo=False)
session = Session(engine0)
stmt = select(Score)

for user in session.scalars(stmt):
    print(user)

Score(Scoreid= '1', Answerid='1', Submissionid='1',Value='2.7',Time='2022-04-11 14:23:09.335798')
Score(Scoreid= '2', Answerid='2', Submissionid='1',Value='9.0',Time='2022-04-11 14:23:09.346955')
Score(Scoreid= '3', Answerid='3', Submissionid='1',Value='6.8',Time='2022-04-11 14:23:09.355259')
Score(Scoreid= '4', Answerid='4', Submissionid='1',Value='1.5',Time='2022-04-11 14:23:09.364528')
Score(Scoreid= '5', Answerid='5', Submissionid='1',Value='4.3',Time='2022-04-11 14:23:09.373052')
Score(Scoreid= '6', Answerid='6', Submissionid='1',Value='10.0',Time='2022-04-11 14:23:09.382879')
Score(Scoreid= '7', Answerid='7', Submissionid='1',Value='0.0',Time='2022-04-11 14:23:09.392705')
Score(Scoreid= '8', Answerid='8', Submissionid='2',Value='4.7',Time='2022-04-11 14:23:09.413627')
Score(Scoreid= '9', Answerid='9', Submissionid='2',Value='9.8',Time='2022-04-11 14:23:09.424257')
Score(Scoreid= '10', Answerid='10', Submissionid='2',Value='3.4',Time='2022-04-11 14:23:09.434610')
Score(Scoreid= '1